In [3]:
import pandas as pd

df = pd.read_csv("SKSNXTR_filtered.csv", index_col=False)

In [ ]:
df[df['종목코드'] == 'KR7069500007']

,종목코드,종목인덱스,체결가격,체결일자,체결시각,시가


In [16]:
import polars as pl

# 스트리밍 스캔 → 앞 20 행만 가져오기
sample = (
    pl.scan_csv("SKSNXTR_filtered.csv")   # 헤더가 이미 있으므로 옵션 생략
)

In [ ]:
sample.filter(pl.col("종목코드") == "KR7069500007").collect()


In [ ]:
sample.filter(pl.col("종목코드") == "KR7229200001").collect()

In [1]:
import FinanceDataReader as fdr

listing = (
    pl.from_pandas(
        fdr.StockListing("KRX")[["ISU_CD", "Name"]]
    )
    .rename({"ISU_CD": "종목코드", "Name": "종목명"})
    .with_columns(pl.col("종목코드").cast(pl.Utf8))
    .unique(subset="종목코드")
    .lazy()
) 

ModuleNotFoundError: No module named 'FinanceDataReader'

In [19]:
listing

In [30]:
result = (
    sample.join(listing, on="종목코드", how="left")
    .drop_nulls("종목명") 
        .collect(streaming=True)
)

/var/folders/bs/f41tt40n4051glsbp8qq036c0000gp/T/ipykernel_97144/2321647073.py:4: DeprecationWarning: The argument `streaming` is deprecated and is being replaced by the `engine` argument.
  .collect(streaming=True)
/var/folders/bs/f41tt40n4051glsbp8qq036c0000gp/T/ipykernel_97144/2321647073.py:4: DeprecationWarning: The old streaming engine is being deprecated and will soon be replaced by the new streaming engine. Starting Polars version 1.23.0 and until the new streaming engine is released, the old streaming engine may become less usable. For people who rely on the old streaming engine, it is suggested to pin your version to before 1.23.0.

More information on the new streaming engine: https://github.com/pola-rs/polars/issues/20947
  .collect(streaming=True)


In [31]:
result

종목코드,종목인덱스,체결가격,체결일자,체결시각,시가,종목명
str,i64,i64,i64,i64,i64,str
"""KR7000020008""",88,9540,20231201,90012057,9540,"""동화약품"""
"""KR7000020008""",88,9540,20231201,90012057,9540,"""동화약품"""
"""KR7000020008""",88,9540,20231201,90012057,9540,"""동화약품"""
"""KR7000020008""",88,9540,20231201,90012057,9540,"""동화약품"""
"""KR7000020008""",88,9540,20231201,90012057,9540,"""동화약품"""
…,…,…,…,…,…,…
"""KYG5307W1015""",6177,2525,20231228,180030021,0,"""엘브이엠씨홀딩스"""
"""KYG5307W1015""",6177,2525,20231228,180030021,0,"""엘브이엠씨홀딩스"""
"""KYG5307W1015""",6177,2525,20231228,180030021,0,"""엘브이엠씨홀딩스"""
